# Thư viện

In [ ]:
import re
import json
import enchant
import pandas as pd
import string
# from pyvi import ViTokenizer, ViPosTagger

# Đối tượng chứa các hàm xử lý dữ liệu

In [ ]:
class tvpl_function:
    def __init__(self):
        self.pattern = 'aàảãáạăằẳẵắặâầẩẫấậeèẻẽéẹêềểễếệiìỉĩíịoòỏõóọôồổỗốộơớờởỡợuùủũúụưừửữứựyỳỷỹýỵđ'

        self.words_en = enchant.Dict("en_US")
        
        self.specialCharset = '!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~'

        self.words = []
        with open("data/words.txt", "r", encoding='utf-8') as file:
            for f in file:
                word = json.loads(f)
                self.words.append(word["text"])

        self.syllables = []
        with open("data/syllable.txt", "r", encoding="utf-8") as file:
            for line in file:
                self.syllables.append(line.strip())
                
        self.mapping={
            'óa':'oá', 'òa':'oà', 'ỏa':'oả', 'õa':'oã', 'ọa':'oạ',
            'óe':'oé', 'òe':'oè', 'ỏe':'oẻ', 'õe':'oẽ', 'ọe':'oẹ',
            'úy':'uý', 'ùy':'uỳ', 'ủy':'uỷ', 'ũy':'uỹ', 'ụy':'uỵ',
            'úâ':'uấ', 'ùâ':'uầ', 'ủâ':'uẩ', 'ũâ':'uẫ', 'ụâ':'uậ',
            'úe':'ué', 'ùe':'uè', 'ủe':'uẻ', 'ũe':'uẽ', 'ụe':'uẹ',
            'úê':'uế', 'ùê':'uề', 'ủê':'uể', 'ũê':'uễ', 'ụê':'uệ',
            'úơ':'uớ', 'ùơ':'uờ', 'ủơ':'uở', 'ũơ':'uỡ', 'ụơ':'uợ',
            'úô':'uố', 'ùô':'uồ', 'ủô':'uổ', 'ũô':'uỗ', 'ụô':'uộ',
            'iá':'ía', 'ià':'ìa', 'iả':'ỉa', 'iã':'ĩa', 'iạ':'ịa',
            'yá':'ýa', 'yà':'ỳa', 'yả':'ỷa', 'yã':'ỹa', 'yạ':'ỵa',
            'uá':'úa', 'uà':'ùa', 'uả':'ủa', 'uã':'ũa', 'uạ':'ụa',
            'ưá':'ứa', 'ưà':'ừa', 'ưả':'ửa', 'ưã':'ữa', 'ưạ':'ựa',
            'ứơ':'ướ', 'ừơ':'ườ', 'ửơ':'ưở', 'ữơ':'ưỡ', 'ựơ':'ượ',
            'íê':'iế', 'ìê':'iề', 'ỉê':'iể', 'ĩê':'iễ', 'ịê':'iệ',
            'ýê':'yế', 'ỳê':'yề', 'ỷê':'yể', 'ỹê':'yễ', 'ỵê':'yệ',
            'uí':'úi', 'uì':'ùi', 'uỉ':'ủi', 'uĩ':'ũi', 'uị':'ụi',
            'aó':'áo', 'aò':'ào', 'aỏ':'ảo', 'aõ':'ão', 'aọ':'ạo',
            'qúa':'quá', 'qùa':'quà', 'qủa':'quả', 'qũa':'quã', 'qụa': 'quạ',
            'Qúa':'Quá', 'Qùa':'Quà', 'Qủa':'Quả', 'Qũa':'Quã', 'Qụa': 'Quạ', 
            'gía':'giá', 'gìa':'già', 'gỉa':'giả', 'gĩa':'giã', 'gịa': 'giạ',
            'Gía':'Giá', 'Gìa':'Già', 'Gỉa':'Giả', 'Gĩa':'Giã', 'Gịa': 'Giạ',
        }

    # chuẩn hóa bảng mã tiếng việt
    # Tạo ra từ điện bộ kí tự
    def load_DictChar(self):
        dic = {}
        char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
            '|')
        charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
            '|')
        for i in range(len(char1252)):
            dic[char1252[i]] = charutf8[i]
        return dic
    
    # Chuyển đổi mã kí tự 1252 sang UTF-8
    def covert_unicode(self, txt):
        dicchar = self.load_DictChar()
        return re.sub(
            r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
            lambda x: dicchar[x.group()], txt) 

    # xóa phần không cần thiết trong văn bản
    def removeUnnecessaryPart(self, row):
        for i, line in enumerate(row.splitlines()):
            if re.match(r"^(PHẦN PHỤ LỤC|DANH MỤC PHỤ LỤC)", line) or re.match(r"^phụ lục", line.lower()) or re.match(r"^(\s|\t)*PHỤ LỤC", line):
                row = '\n'.join(row.splitlines()[:i])
                break     
        
        return row
    
    # đổi về chữ thường
    def text_lower(self, text):
        return text.lower()
    
    #  Tìm và thay thế các ký tự không in được trong text thành khoảng trắng
    def  replace_non_printable_chars(self, row):
        return  ''.join(char if char in string.printable or char.lower() in self.pattern else ' ' for char in row)
                
    def data_processing(self, row):
        row = self.covert_unicode(row)                          # Chuẩn hóa bảng mã kí 
        row = self.removeUnnecessaryPart(row)                   # Loại bỏ phần không cần thiết
        row = self.replace_non_printable_chars(row)             # Thay thế các chuỗi không in được thành khoảng trắng
        row = self.text_lower(row)                              # Đổi thành chữ thường
        
        result_line = []
        for line in row.splitlines():           
            if line != ' ' and line != '':
                line = line.replace('\n', '').replace('\t', '')                 # Loại \n và \t trong dòng
                line = self.Handling_Stuck_Syllables(line)                      # Tách âm tiết bị dính nhau
                line = ViTokenizer.tokenize(line).lstrip().rstrip()             # Tách câu, tách từ tiếng 
                result_line.append(line)
        row = "\n\n".join(result_line)

        row = row.replace(" / ", "/")
        row = row.replace(" - ", "-")

        return row
        return row

    # Chuẩn hóa y và i (vd: tỷ lệ -> tỉ lệ, bác sỹ -> bác sĩ)
    def chuan_hoa_y_i(self, word):
        set_yi ={1: 'yýỳỷỹỵ', 2: 'iíìỉĩị'}
        index_yi = None
        index_setyi = None
        for i in range(6):
            if set_yi[1][i] in word or set_yi[2][i] in word:
                index_yi = word.index(set_yi[1][i]) if set_yi[1][i] in word else word.index(set_yi[2][i])
                index_setyi = i
                break

        if index_yi is not None:
            if len(word) == 1 and word in set_yi[2]:
                word = set_yi[1][index_setyi]
            elif (len(word) > 1 and index_yi == len(word) - 1 and word[index_yi] in set_yi[1] 
            and word[:index_yi] in ['b','c','ch','d','đ','g','gh','h','k','kh','l','m','n','ng','ngh','nh','p','ph','r','s','t','th','tr','v','x']):
                word = word[:index_yi] + set_yi[2][index_setyi]

        return word

    # Xử lý các âm tiết bị lỗi dính nhau trong từng câu của văn bản
    def Handling_Stuck_Syllables(self, sentence):
        result_sentence = []
        sentence = self.split_speicalChar(sentence) # Tách các kí tự đặc biệt (ví dụ: 'luật.' -> 'luật .')
        for word in sentence.split():       # xét từng chữ
            word = self.standardize_Tone(word)  # Chuẩn hóa thanh điệu
            word = self.chuan_hoa_y_i(word)     # Chuẩn hóa y/i
            # Nếu từ không chứa kí tự, từ có nghĩa, là số ký hiệu hoặc là từ tiếng anh thì bỏ qua
            if (len(word) == 1
                    or any(char in word for char in self.specialCharset)
                    or any(char in word.lower() for char in ["f","j","w","z"])
                    or any(char in '0123456789' for char in word)
                    or self.words_en.check(word) 
                    or self.syllable_check(word) 
                    ):
                result_sentence.append(word)
                continue

            # Từ có nghĩa nhưng có 2 chữ giống nhau liên tiếp (vd: hoaang, luuật)
            word = self.double_char(word)
            if self.syllable_check(word):   # xét từ có nghĩa chưa
                result_sentence.append(word)
                continue

            check = True
            #Kiểm tra cặp từ có nghĩa trong từ điển
            for i in range(1, len(word)-1):
                text = word[:i] + " " + word[i:]
                if self.dictionary_check(text):
                    check = False
                    break
                
            # Nếu không có cặp từ có nghĩa
            if check:
                text = self.split_syllable(word)   # text = text (được tách ra)/ False (không được tách)

            # Xử lý các âm tiết bị lỗi dính nhau (các trường hợp chữ viết tắt và sai chính tả -> False)
            result_sentence.append(text if text else word) # nếu False thì từ được giữ nguyên

        return ' '.join(result_sentence)

    # Tách các kí tự đặc biệt 
    def split_speicalChar(self, text):  
        for i in self.specialCharset:
            if i in text:
                text = text.replace(i, f" {i} ")
        return text

    # Trả kết quả xử lý tách các âm tiết dính nhau
    def split_syllable(self, syllable):
        
        memories = []
        while syllable != '':
            text_len = len(syllable)    # Độ dài từ hiện tại
            memory = []                 # Chứa các từ có nghĩa 
            queue = ''                  # Cập nhật các từ

            for i in range(text_len):
                # Xử lý từ có nghĩa (vd: nóng -> memory = [nó, nón, nóng])
                queue += syllable[i]
                
                # Lọc ra cá kí tự đứng 1 mình, trừ các từ có thể
                if len(queue) == 1 and queue not in ['ả', 'ế', 'ô', 'ố', 'ổ', 'y', 'ý', 'ỷ', 'ủ']:
                    continue

                if self.syllable_check(queue) or self.dictionary_check(queue):
                    memory.append(queue)
            
            # Nếu memory có chứa các từ có nghĩa
            if memory:
                memories.append(memory)
                syllable = syllable[len(memory[-1]):]     # Loại bỏ từ có nghĩa trong chuỗi syllable và xét tiếp theo vòng lặp while
            # Nếu không
            else:
                # Nếu xét cả chuỗi không có từ nào hoặc chỉ 1 từ có nghĩa trong chuỗi -> False
                if memories and len(memories[0]) > 1:
                    # kiểm tra chuỗi từ trước có 2 từ trở lên thì ghép chữ cuối cùng của chuỗi từ đó [-1]  
                    for j in range(len(memories), 0, -1):
                        # Vd: hoặcđiago -> memories = [['cá', 'các', 'cách'],['i']] --- syllable = 'ệp'
                        if len(memories[j-1]) >= 2:
                            syllable = memories[j-1].pop(-1)[-1] + ''.join(c[-1] for c in memories[j:]) + syllable
                            memories = memories[:j]
                            break
                else:
                    return False
        
        return ' '.join(i[-1] for i in memories)

    # kiểm tra từ hoặc cụm từ có trong từ điển
    def dictionary_check(self, text):
        return text.lower() in self.words
    
    # kiểm tra âm tiết có trong từ điển
    def syllable_check(self, syllable):
        return syllable.lower() in self.syllables

    # xóa các kí tự lặp (trừ "o")
    def double_char(self, word):
        result = word[0]  # Khởi tạo result với ký tự đầu tiên của word
        for i in range(1, len(word)):  # Bắt đầu vòng lặp từ chỉ số 1
            if result[-1] == word[i] and word[i] != 'o':
                continue
            result += word[i]

        return result

    # Chuẩn hóa thanh điệu 
    def standardize_Tone(self, word):  
        if word != "gịa" and word != "quốc":
            for key, value in self.mapping.items():
                word = word.replace(key, value)
        return word

# Tải dữ liệu

In [ ]:
PATH_DATA = 'data/tvpl.csv'
df_raw = pd.read_csv(PATH_DATA, encoding='utf-8-sig')

# Định dạng cột 'Ngày ban hành'
df_raw['Ngày ban hành'] = pd.to_datetime(df_raw['Ngày ban hành'], format='%d/%m/%Y').dt.strftime('ngày %d tháng %m năm %Y')  # định dạng cột 'ngày ban hành'

# Loại bỏ các hàng ngoại lệ
list_remove = [9,369,2936,3789,4630,5740,6707]                      
df_raw = df_raw.drop(index=list_remove).reset_index(drop=True)

vb = tvpl_function()

column_vb = 'Nội dung'
column_qh = 'Mối quan hệ'

df_raw.info()
df_raw.head()

# Kiểm tra giá trị trống

In [ ]:
print(df_raw.isna().values.any(), df_raw.isnull().values.any())
df = df_raw.copy()
df.info()

# Xử lý văn bản

In [ ]:
df[column_vb] = df[column_vb].apply(vb.data_processing)

# Đặt tên theo cấu trúc '[loại văn bản] + [số hiệu] + [ngày ban hành] + [cơ quan ban hành]' ở dòng đầu tiên của mỗi văn bản
for i in range(len(df)):
    title = df.loc[i, 'Loại văn bản'] + ' số ' + df.loc[i, 'Số hiệu'] + ' ' + df.loc[i, 'Ngày ban hành'] + ' của ' + df.loc[i, 'Cơ quan ban hành'] 
    df.at[i, column_vb] = title + '\n\n' + df.at[i, column_vb]

type_documents = {
    ' Hiến_pháp ': 'HP',
    ' Bộ_luật ': 'BL',
    ' Luật ': 'LT',
    ' Pháp_lệnh ': 'PL',
    ' Lệnh ': 'LH',
    ' Nghị_quyết ': 'NQ',
    ' Nghị_quyết liên_tịch ': 'NQLT',
    ' Nghị_định ': 'NĐ',
    ' Quyết_định ': 'QĐ',
    ' Thông_tư ': 'TT',
    ' Thông_tư liên_tịch ': 'TTLT',
    ' Chỉ_thị ': 'CT',
}
# Xử lý khác
for i in range(len(df)):
    # Xử lý cột 'Nội dung' (in hoa các đối tượng)
    row_vb = df.loc[i, column_vb]
    for key, value in type_documents.items():
        row_vb = row_vb.replace(key.replace(' ', '').lower(), key)             # viết hoa đầu chữ các tên loại văn bản
    df.at[i, column_vb] = row_vb

    # Xử lý cột "Mối quan hệ" (chuẩn hóa cột 'Mối quan hệ')
    row_qh = df.loc[i, column_qh]
    row_qh = vb.covert_unicode(row_qh).replace('“', '"').replace('”', '"')
    row_qh = ViTokenizer.tokenize(row_qh).replace(" / ", "/").replace(" - ", "-")
    row_qh = row_qh.replace('bị sửa_đổi , bổ_sung', 'được sửa_đổi , bổ_sung')
    row_qh = {key.lstrip().rstrip().capitalize(): [v.lstrip().rstrip() for v in value] for key, value in eval(row_qh).items()}
    df.at[i, column_qh] = row_qh



# Xuất file csv
df.to_csv("data/tvpl_processing.csv", encoding="utf-8-sig", index=False)
df.head()

# Thống kê các tên văn bản có trong văn bản chính

In [ ]:

dict_type = {}
for i, row in enumerate(df[column_vb]):
    for key, value in type_documents.items():
        count = row.count(key)
        if key not in dict_type:
            dict_type[key] = count
        else: 
            dict_type[key] = dict_type[key] + count

print(len(dict_type))
dict_type

# Xác định các nhãn quan hệ trong tập dữ liệu

In [ ]:

label_list = []

for i, row_dict in enumerate(df[column_qh]):
    for key in row_dict.keys():
        if key not in label_list:
            label_list.append(key)

label_list = [key for key in label_list]
label_list

# Gán nhãn

In [24]:
# import re
# import pandas as pd

df = pd.read_csv('data/tvpl_processing.csv', encoding='utf-8-sig')
# df1 = df.copy()

column_vb = 'Nội dung'
column_qh = 'Mối quan hệ'

label_dict = {
    'Được sửa_đổi , bổ_sung': 'DSD',
    'Bị thay_thế': 'BTT',
    'Dẫn chiếu': 'DC',
    'Căn_cứ': 'CC',
    'Được hướng_dẫn': 'DHD',
    'Hết hiệu_lực': 'HHL',
}
object_dict = {
    ' Hiến_pháp ': 'HP',
    ' Bộ_luật ': 'BL',
    ' Luật ': 'LT',
    ' Pháp_lệnh ': 'PL',
    ' Lệnh ': 'LH',
    ' Quyết_định ': 'QĐ',
    ' Nghị_định ': 'NĐ',
    ' Nghị_quyết ': 'NQ',
    ' Nghị_quyết liên_tịch ': 'NQLT',
    ' Thông_tư ': 'TT',
    ' Thông_tư liên_tịch ': 'TTLT',
    ' Chỉ_thị ': 'CT',
}

df[column_qh] = df.apply(lambda x: eval(x[column_qh]), axis=1)


In [ ]:

def auto_label(text_str, key, value, row_qh, label_dict):
    label = text_str
    # pattern = rf'({key})' + r' ((?:(?!ngày|số|này|tại|theo|thì|và)\w+[ _])+)?(số)? (\d+/\d+/[a-zđ]+(?:\d+|[-\w]+)+|(?:\w+[-]\d+(?:/[\w\d]+)+|(?:\d+[-]\w+)+|(?:\d+[/]\w+(?:[-\w]+)?)+|\d+)?( ngày \d{1,2}(?: tháng |[-]|[/])\d{1,2}(?: năm |[-]|[/])\d{4})?'
    pattern = rf'({key})' + r' ((?:(?!ngày|số|này|tại|theo|thì|và|[A-ZĐ])\w+[ _])+)?(số)?\s?(\d+?[\w]+(?:(?:/|-)+(?:\d|\w)+)+)?\s?(ngày \d{1,2}(?: tháng |[-]|[/])\d{1,2}(?: năm |[-]|[/])\d{4})?'
    # pattern = rf'({key})((?:(?!ngày|số|này|tại|theo|thì)[^.;()]+\w+[ _])+)?(số)? (\d+/\d+/[a-zđ]+(?:\d+|[-]\w+)+|(?:\w+[-]\d+)+|(?:\d+[-]\w+)+|\d+)?( ngày \d{1,2}(?: tháng |[-/])\d{1,2}(?: năm |[-/])\d{4})?'
    list_objects = re.findall(pattern, text_str)
    list_objects = list(set(list_objects))
    if list_objects:     
        for object in list_objects:
            object_tuple = tuple(filter(lambda x: x != '', object))
            if len(object_tuple) <= 2:
                continue
            object = ' '. join(object_tuple).replace('  ', ' ')
            if 'này' in object or 'của' in object or object + ' ngày' in text_str:
                continue
            
            if (key == 'Nghị_quyết' or key == 'Thông_tư') and ('Thông_tư liên_tịch' in object or 'Nghị_quyết liên_tịch' in object):
                continue
            qh = check_relationship(text_str, object, row_qh, label_dict)
            object = f' {object} '
            label_object = f' <{value} rel="{qh}">' + object + f"</{value}> "
            
            label = label.replace(object, label_object)

    return label

# check có trong quan hệ
def check_relationship(text_str, object, row_qh, label_dict):
    cc_check = ['căn_cứ ' + object, 'căn_cứ vào ' + object]
    if  any(cc.lower() in text_str.lower() for cc in cc_check):
        return 'CC'
    
    pattern = r'(tại|theo)+ ([^.;]+ )?'+rf'{object}'
    if re.search(pattern, text_str):
        return 'DC'
    
    pattern = rf'{object}' + r'( [^.;]+)? được sửa_đổi , bổ_sung '
    if re.search(pattern, text_str):
        return 'DSD'

    
    pattern = rf'{object}' + r'( [^.;]+)? hết hiệu_lực '
    if re.search(pattern, text_str):
        return 'HHL'

    for key, list_value in row_qh.items():
        for value in list_value:
            key_element = re.findall(r'\d+/\d+/\w+(?:-\w+)?|\d+/\w+(?:-\w+)?|\d+-\w+', value)
            if key_element:
                key_element = key_element[0]
            else:
                key_element = re.sub(r'\d+', '', value)
            if key_element.lower() in object:
                return label_dict[key]
    return 'None'


# text = 'căn_cứ Nghị_định số 126/2020/nđ-cp ngày 19 tháng 10 năm 2020 của chính_phủ quy_định chi_tiết một_số điều của Luật quản_lí thuế'
# key = 'Nghị_định'
# value = 'NĐ'
# print(auto_label(text, key, value, {}, label_dict))


In [25]:

df1 = df.copy()
for i in range(len(df1)):
# for i in range(100):
    row_vb = df1.loc[i, column_vb]
    row_qh = df1.loc[i, column_qh]

    line_total = []
    line_total.extend(row_vb.splitlines()[0:2])
    for line in row_vb.splitlines()[2:]:
        for key_object, value_object in object_dict.items():
            if key_object in line:    
                key_o = key_object.strip()
                if key_o == 'Hiến_pháp':
                    pattern = rf'({key_o}) (nước cộng_hoà xã_hội chủ_nghĩa việt_nam)? (năm \d+|\d+)?'
                    objects = re.search(pattern, line)
                    if objects:
                        objects = objects[0]
                        qh = check_relationship(line, objects, row_qh, label_dict)
                        label_object = f'<{value_object} rel="{qh}">' + ' ' + objects + ' ' + f"</{value_object}>"
                        line = line.replace(objects, label_object)
                else:
                    line = auto_label(line, key_o, value_object, row_qh, label_dict)
                     
        line_total.append(line)

    df1.at[i, column_vb] = '\n'.join(line_total)


# Export data
_df = df1
for i in range(len(_df)):
    with open(f'data/source_test/vb_{i}.xml', 'w', encoding='utf-8') as file:
        for line in _df.loc[i, column_vb].splitlines():
            file.write(line + '\n')
        
    
# with open('text.txt', 'w', encoding='utf-8') as file:
#     for line in _df.loc[2, column_vb].splitlines():
#         file.write(line + '\n')

# Xuất file csv
# df1.to_csv("data/tvpl_label.csv", encoding="utf-8-sig", index=False)